# Chapter 4: Inputs, Buttons & Control Systems 🎛️

We have mastered output (LEDs). Now we need **Input**.
Without input, your Pico is just a traffic light. With input, it becomes a *game controller*, a *keyboard*, or a *security system*.

## The Switch: The Simplest Sensor
A button is just two wires that touch when you press them. 
But electronically, it's tricky. If a pin isn't connected to anything (floating), is it 0 or 1? 
*Answer: It's random noise from the universe. Bad.* 

We need to tie it down. 

![Pull Up vs Pull Down](assets/button_logic_pullup_pulldown.png)

### The Easy Way: Internal Pull-Ups
Luckily, the RP2040 has built-in resistors we can turn on with code. We almost always use **PULL_DOWN** (Standard for beginners) or **PULL_UP** (Standard for pros/industrial).

## 1. Wiring a Button

Let's consult your shiny new Pinout Diagram:
![Pinout](assets/raspberry_pi_Pico-R3-Pinout.png)

**Wiring:**
- 3.3V Pin -> One side of Button
- GP15 -> Other side of Button

*Note: If we use PULL_DOWN in code, we connect the button to 3.3V. When pressed, 3.3V rushes in (HIGH). When released, the resistor pulls it to 0V (LOW).*

In [ ]:
import machine
import time

# Setup the LED
led = machine.Pin("LED", machine.Pin.OUT)

# Setup the Button on GP14
# machine.Pin.IN tells the Pico to LISTEN
# machine.Pin.PULL_DOWN tells it "If no one is pressing, assume 0 Volts"
button = machine.Pin(14, machine.Pin.IN, machine.Pin.PULL_DOWN)

print("Button Ready on GP14. Press it!")

## 2. Reading the State

The simplest way is `button.value()`. 
- 1 = Pressed
- 0 = Not Pressed

In [ ]:
# The Doorbill Logic
# LED is ON only while Button is HELD

while True:
    if button.value() == 1:
        led.value(1)
        print("Ding-Dong!", end="\r") # \r keeps it on one line
    else:
        led.value(0)
    
    time.sleep(0.1) # Debounce/CPU saver

### ⏸️ Pause & Play: The Toggle Switch
The code above is a "Momentary" switch (like a car horn).
Can you change it to a "Toggle" switch (like a light switch)?

*Logic: If button gets pressed, flip the LED state. But wait until button is released before checking again, or it will flicker madness.*

In [ ]:
# Toggle Logic Attempt
led_state = False

while True:
    if button.value() == 1:
        # Toggle our variable
        led_state = not led_state 
        
        # Write to LED
        led.value(led_state)
        
        print(f"Light is now: {led_state}")
        
        # CRITICAL: Wait for user to let go, otherwise it loops 1000 times a second
        while button.value() == 1:
            time.sleep(0.01)
            
    time.sleep(0.05)

## 3. Project: Dimmable Light (PWM + Buttons)

Let's combine Chapter 2 and Chapter 4.
Use the button to cycle through brightness levels: Off -> Low -> Med -> High -> Off.

In [ ]:
# Setup PWM LED again
pwm_led = machine.PWM(machine.Pin("LED", machine.Pin.OUT))
pwm_led.freq(1000)

brightness_levels = [0, 5000, 32000, 65535]
current_level_index = 0

print("Click to change brightness!")

while True:
    if button.value() == 1:
        # Move to next level
        current_level_index += 1
        
        # Wrap around if we go past the end
        if current_level_index >= len(brightness_levels):
            current_level_index = 0
            
        # Apply brightness
        new_duty = brightness_levels[current_level_index]
        pwm_led.duty_u16(new_duty)
        
        print(f"Brightness: {new_duty}")
        
        # Wait for release
        while button.value() == 1:
            time.sleep(0.01)
            
    time.sleep(0.05)

## 🎓 Challenge: Two-Button Game Show

If you have two buttons (connect 2nd one to GP15):
Make a game where the LED starts OFF.
- Button A turns it **RED**.
- Button B turns it **BLUE**.
- Both together turn it **PURPLE**.

*Hint: You'll need `if buttonA.value() and buttonB.value():`*